In [18]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("..")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
import torch
from calflops import calculate_flops
import sophius.utils as utils
import sophius.dataload as dload
from sophius.modelgen import ConvModelGenerator
from sophius.train import train_express_gpu
import torchvision.datasets as dset
import torchvision.transforms as T
from sophius.encode import Encoder
from sophius.utils import calc_model_flops
import sqlite3
import pandas as pd
import seaborn as sns
from tqdm import tqdm

VAL_SIZE = 1024

cifar10 = dset.CIFAR10('../data/CIFAR10', train=True, download=True, transform=T.ToTensor())
cifar_gpu = dload.cifar_to_gpu(cifar10)
loader_gpu = dload.get_loader_gpu(cifar_gpu, val_size=VAL_SIZE, batch_size=1024)

encoder = Encoder()

Files already downloaded and verified


In [20]:
model_gen = ConvModelGenerator((3, 32, 32), 10, conv_num=6, lin_num=3)
model_tmpl = model_gen.generate_model_tmpl()
print(model_tmpl)
fixed_model_gpu = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)

print(encoder.model2vec(model_tmpl), end='\n\n')

print(encoder.model2hash(model_tmpl))

Conv2d       (96, 27, 27)   (6, 6)   (1, 1)  
PReLU        (96, 27, 27)  
BatchNorm2d  (96, 27, 27)  
AvgPool2d    (96, 14, 14)   (3, 3)   (2, 2)  
Conv2d       (128, 7, 7)    (7, 7)   (2, 2)  
PReLU        (128, 7, 7)   
MaxPool2d    (128, 2, 2)    (2, 2)   (3, 3)  
BatchNorm2d  (128, 2, 2)   
Flatten      512           
Linear       512           
LeakyReLU    512            (0.1)   
Linear       2048          
PReLU        2048          
Linear       10            

[[0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 0 0 1]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 1 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

In [21]:
model_tmpl = model_gen.generate_model_tmpl()
arr = encoder.model2vec(model_tmpl)

hex_str = encoder.model2hash(model_tmpl)

print(arr)
print()
print(hex_str)

[[0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 1 0 0 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 1 0 0 0 0 0 1 0 0 1]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 1 0 0 0 1 0]
 [0 0 0 0 1 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 1 0 0 0 1 0 1 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 0 0 1 1 0 1 0 0 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 1 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 

In [22]:
df = pd.DataFrame(columns=['hash', 'flops', 'time', 'val_acc', 'train_acc'])

for i in tqdm(range(10)):
    model_tmpl = model_gen.generate_model_tmpl()
    model = model_tmpl.instantiate_model().type(torch.cuda.FloatTensor)
    flops = calc_model_flops(model, (3, 32, 32))
    # express train
    t, val_acc, train_acc = train_express_gpu(model = model,
                                              train = True,
                                              loader = loader_gpu,
                                              milestones = [],
                                              num_epoch = 1,
                                              verbose = False)
    
    hash_str = encoder.model2hash(model_tmpl)
    
    df.loc[i] = [hash_str, flops, t, val_acc, train_acc]
    
    with sqlite3.connect('../data/models.db') as conn:
        df.loc[i:i].to_sql('test', conn, if_exists='append', index=False)
    
    # print(df.loc[i])

100%|██████████| 10/10 [01:54<00:00, 11.43s/it]


In [23]:
df

,hash,flops,time,val_acc,train_acc
0,0084006208080000004460000080406220000000400000...,31651232,1.986887,0.345703,0.333984
1,008084061002000000485000100200004000000001000000,370080,0.145512,0.418945,0.423828
2,0082080a2000000000432000400000000081010a080400...,2153696,0.453577,0.458984,0.477539
3,0080209210020000400000000080821210040000004260...,60606496,3.134847,0.125977,0.117188
4,0081006220000000004310004000000000234800008402...,132548264,22.475040,0.299805,0.325195
5,0080420620000000004860004000000000803005080800...,1289152,0.376307,0.191406,0.172852
6,00840809100400000022a8000010000001000000,52848,0.150514,0.134766,0.083984
7,0084100908040000008400612000000040000000010000...,749376,0.408250,0.231445,0.221680
8,008028211002000040000000008100a220000000008041...,1018891504,84.013951,0.397461,0.390625
9,0080408a08040000004890004000000000233000008804...,3768096,0.503587,0.347656,0.373047


In [6]:
import sqlite3
import pandas as pd

with sqlite3.connect('../data/models.db') as conn:
    df = pd.read_sql('SELECT * FROM conv_10_lin_3_epoch_50', conn)
df

,hex_str,flops,macs,params,time,val_acc,train_acc
0,0080281240000000004320002000000000810109004460...,18070912,8890624,857034,92.446377,0.686523,0.941406
1,00808806004260000024a8004000000008040000008040...,1656288,818304,728619,65.467208,0.444336,0.464844
2,0088080908080000400000000088020600426000200000...,614096,304480,286370,17.895498,0.488281,0.528320
3,00820051400000000028b0001004000000804211200000...,3273152,1622368,639818,14.079044,0.596680,0.784180
4,00804062200000000024a8000080209240000000002328...,152657664,76097536,1112522,200.821419,0.630859,0.686523
...,...,...,...,...,...,...,...
995,008808110029300010040000004850000082004a080400...,17151952,8568320,8513108,36.679291,0.468750,0.538086
996,00810411100800000081020a4000000000485000080800...,6080256,3020480,341002,15.669035,0.630859,0.875977
997,0081022220000000004460000084020a00451000080400...,7427008,3608384,20475,31.284352,0.470703,0.547852
998,0080408508080000004260004000000000840112080400...,2878912,1427968,613947,20.990275,0.572266,0.751953


In [19]:
df.sort_values(by='val_acc', ascending=False, inplace=True)

In [20]:
from sophius.encode import Encoder, str_to_vec
from sophius.templates import ModelTmpl_, LinearTmpl

encoder = Encoder()

def decode(arr):
    templates = []
    for vec in arr:
        templates.append(encoder.decode_template(vec))
    templates.append(LinearTmpl())
    return ModelTmpl_((3, 32, 32), 10, *templates)
    

df['models'] = df['hex_str'][:20].apply(str_to_vec).apply(decode)

In [21]:
df[['models', 'val_acc', 'flops']].dropna()

,models,val_acc,flops
930,"Conv2d (96, 9, 9) (4, 4) (2, 2) \...",0.813477,961733248
302,"Conv2d (64, 17, 17) (4, 4) (2, 2) \...",0.787109,275556800
23,"Conv2d (96, 11, 11) (4, 4) (2, 2) \...",0.755859,48945472
763,"Conv2d (8, 11, 11) (4, 4) (2, 2) \...",0.752930,212892960
310,"Conv2d (32, 31, 31) (4, 4) (2, 2) \...",0.726562,12470656
77,"Conv2d (192, 7, 7) (4, 4) (2, 2) \...",0.726562,24060928
501,"Conv2d (192, 11, 11) (4, 4) (2, 2) \...",0.725586,54068064
946,"Conv2d (192, 11, 11) (4, 4) (2, 2) \...",0.719727,19730784
359,"ReLU (128, 9, 9) \nBatchNorm2d (128...",0.719727,55969408
873,"Conv2d (96, 11, 11) (4, 4) (2, 2) \...",0.715820,34535904


In [41]:
for i, row in df[['models', 'val_acc', 'flops']].dropna()[:10].iterrows():
    print(f"val acc: {row['val_acc']:.3f}", f"\t\tflops: {row['flops'] // 1024 // 1024}M", end='\n\n')
    print(row['models'])
    print("="*80)

val acc: 0.813 		flops: 917M

Conv2d       (96, 9, 9)     (4, 4)   (2, 2)  
ReLU         (96, 9, 9)    
BatchNorm2d  (96, 9, 9)    
Conv2d       (96, 3, 3)     (4, 4)   (2, 2)  
PReLU        (96, 3, 3)    
BatchNorm2d  (96, 3, 3)    
Conv2d       (96, 1, 1)     (4, 4)   (2, 2)  
LeakyReLU    (96, 1, 1)     (0.001) 
AvgPool2d    (96, 1, 1)     (4, 4)   (2, 2)  
Conv2d       (96, 1, 1)     (4, 4)   (2, 2)  
BatchNorm2d  (96, 1, 1)    
AvgPool2d    (96, 1, 1)     (4, 4)   (2, 2)  
PReLU        (96, 1, 1)    
Conv2d       (96, 1, 1)     (4, 4)   (2, 2)  
LeakyReLU    (96, 1, 1)     (0.001) 
AvgPool2d    (96, 1, 1)     (4, 4)   (2, 2)  
Conv2d       (96, 1, 1)     (4, 4)   (2, 2)  
GlobalAvgPool2d (96, 1, 1)    
Flatten      96            
Linear       10            

val acc: 0.787 		flops: 262M

Conv2d       (64, 17, 17)   (4, 4)   (2, 2)  
BatchNorm2d  (64, 17, 17)  
ReLU         (64, 17, 17)  
Conv2d       (64, 9, 9)     (4, 4)   (2, 2)  
LeakyReLU    (64, 9, 9)     (0.001) 
Conv2d     